In [1]:
%%capture
!pip install rio-tiler==4.1.11 loguru

# Xarray Tile Testing

In [2]:
%load_ext autoreload
%autoreload
from xarray_tile_test import XarrayTileTest
import sys; sys.path.append('..')
import helpers.eodc_hub_role as eodc_hub_role
import warnings
warnings.filterwarnings("ignore")

In [3]:
credentials = eodc_hub_role.fetch_and_set_credentials()

In [5]:
xarray_tile_test = XarrayTileTest(
    dataset_id='cmip6-kerchunk',
    dataset_url='s3://nasa-eodc-data-store/test-data/cmip6-kerchunk/combined_CMIP6_daily_GISS-E2-1-G_tas_kerchunk.json',
    variable='tas',
    extra_args={'reference': True},
)

In [ ]:
xarray_tile_test.array_specs

In [ ]:
xarray_tile_test.run_batch({'zoom': 0})

In [ ]:
xarray_tile_test.store_results(credentials)

## Run many tests

In [ ]:
# TODO

## Read results

In [ ]:
# TODO